# **Introduction**

This notebook documents all the measurements we created and the meanings and interpretations behind them. We use the normal times as examples.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
# suppress warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns
# avoid scientific digit
# pd.options.display.float_format = '{:.2f}'.format

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
link1 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times_svi.csv'
link2 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_covid_svi.csv'
normal = pd.read_csv(link1)
covid = pd.read_csv(link2)

In [ ]:
normal.shape, covid.shape
# correct

((66397, 104), (5386, 103))

In [ ]:
covid.columns[-13:]

Index(['average_year_eviction_count_per_unit(building)',
       'average_year_eviction_count(building)',
       'average_year_eviction_count_per_nta(nta)',
       'evictions_per_nta_1kpop(nta)', 'buildings_affected_per_nta',
       'unit_per_nta', 'evictions_per_1kunit_nta_1kpop(nta)',
       'children_impacted_nta_per_1kpop', 'unemployed_impacted_nta_per_1kpop',
       'elderly_impacted_nta_per_1kpop', 'bh_impacted_nta_per_1kpop',
       'hburd_impacted_nta_per_1kpop',
       'average_year_eviction_per_building_nta(nta)'],
      dtype='object')

In [ ]:
# evictions per nta normalized by population
55.4/1523*1000, \
# evictions per unit (in an nta) normalized by population (*1000, *1000) double normalized for standardization
55.4/1523/39480.0*1000*1000
# correct

0.9213671358653966

In [ ]:
normal[['primary_key', 'nta', 'e_totpop','borough', 'unit_per_nta', 'evictions_per_nta_1kpop(nta)', 'average_year_eviction_count(building)', 'average_year_eviction_count_per_unit(building)',
        'average_year_eviction_count_per_nta(nta)', 'average_year_eviction_per_building_nta(nta)', 'evictions_per_1kunit_nta_1kpop(nta)']].sort_values(by='evictions_per_nta_1kpop(nta)', ascending=False).head(10)

,primary_key,nta,e_totpop,borough,unit_per_nta,evictions_per_nta_1kpop(nta),average_year_eviction_count(building),average_year_eviction_count_per_unit(building),average_year_eviction_count_per_nta(nta),average_year_eviction_per_building_nta(nta),evictions_per_1kunit_nta_1kpop(nta)
14692,309061/22_210917,Lenox Hill-Roosevelt Island,1523.0,MANHATTAN,39480.0,36.375575,0.6,0.002857,55.4,2.946809,0.921367
1756,084970/16_202073,Lenox Hill-Roosevelt Island,1523.0,MANHATTAN,39480.0,36.375575,0.6,0.002857,55.4,2.946809,0.921367
39043,64622/19_493494,Lenox Hill-Roosevelt Island,1523.0,MANHATTAN,39480.0,36.375575,0.6,0.002857,55.4,2.946809,0.921367
8027,29668/17_335287,Crotona Park East,4292.0,BRONX,2280.0,26.281454,0.2,0.016667,112.8,5.936842,11.526953
17201,313573/23_132495,East Concourse-Concourse Village,11280.0,BRONX,6496.0,25.496454,0.4,0.025000,287.6,7.083744,3.924947
23092,32834/16_55430,East Concourse-Concourse Village,11280.0,BRONX,6496.0,25.496454,0.4,0.025000,287.6,7.083744,3.924947
8317,300157/20_34280,Central Harlem North-Polo Grounds,19138.0,MANHATTAN,147004.0,17.055074,5.0,0.019455,326.4,5.706294,0.116018
8449,300282/21_33010,Central Harlem North-Polo Grounds,19138.0,MANHATTAN,441012.0,17.055074,3.4,0.004410,326.4,5.706294,0.038673
6753,250762/18_82775,Central Harlem North-Polo Grounds,19138.0,MANHATTAN,441012.0,17.055074,3.4,0.004410,326.4,5.706294,0.038673
6722,250616/19_95114,Central Harlem North-Polo Grounds,19138.0,MANHATTAN,441012.0,17.055074,4.8,0.006226,326.4,5.706294,0.038673


In [ ]:
normal.head()

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.9140,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,0.8,266.0,2.747905,725,2175.0,1.263405,0.618279,0.305018,0.379211,2.437392,1.228314,3.668966
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.635941,-74.011883,7.0,38.0,118.0,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),1920.0,B2,2.0,2.0,"A.R.M. PARKING, LLC",1204.0,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,0.300000,0.6,38.0,0.408567,136,272.0,1.502085,0.103776,0.030643,0.053522,0.174050,0.178135,2.794118
2,*324973/22_5308,3057820030,*324973/22,5308,462 60TH STREET,FOURTH FLOOR APT AKA,2024-08-13,BROOKLYN,11220,Not an Ejectment,Possession,40.640008,-74.017068,7.0,38.0,122.0,3143435,Sunset Park West,2024,2024-08,POINT (-74.017068 40.640008),1907.0,C3,4.0,4.0,"LIN, RONG LAN",4800.0,pre-war,walk-up,False,mid-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",3-5 units,False,medium-small,Q4 (largest 25%),1900-1909,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0,0.150000,0.6,39.6,0.425770,147,588.0,0.724098,0.108146,0.031933,0.055776,0.181378,0.185636,2.693878
3,*53336/16_170279,2032510420,*53336/16,170279,

# **1. average_year_eviction_count(building)**

This measurement measures the average eviction per building (aggregated by all the buildings affected by evictions)

- group by bin to get total evictions per building.
- divide by the number of years (5 for noaml, 3 for covid)
- for example, if the result is 6, it means on average, this building has 6 evictions per year during normal years.

$$
\mathrm{average\ year\ eviction\ count}(\mathrm{building}) = \frac{\mathrm{Total\ Evictions}_{\mathrm{building}}}{\mathrm{Number\ of\ Years}_{\mathrm{observed}}}
$$


In [ ]:
normal[['bin', 'nta', 'average_year_eviction_count(building)']]

,bin,nta,average_year_eviction_count(building)
0,3083989,East New York,0.8
1,3143881,Sunset Park East,0.6
2,3143435,Sunset Park West,0.6
3,2015444,Van Cortlandt Village,0.8
4,2003900,Mott Haven-Port Morris,1.6
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,3.8
66393,5101137,Westerleigh,0.4
66394,5101137,Westerleigh,0.4
66395,5108656,West New Brighton-New Brighton-St. George,0.2


# **2. average_year_eviction_count_per_unit(building)**

This measurement measures the average eviction per building per unit(aggregated by all the buildings affected by evictions and then divide by the units in each building)

- an example would be: if the result is 0.2, it means on average per year, each unit in this building has 0.2 evictions. Or every 5 years, this building has one eviction.


$$
\text{average year eviction count per unit}(\text{building}) = \frac{\text{average year eviction count}(\text{building})}{\text{units in building}}
$$


In [ ]:
normal[['bin', 'nta', 'average_year_eviction_count_per_unit(building)']]

,bin,nta,average_year_eviction_count_per_unit(building)
0,3083989,East New York,0.266667
1,3143881,Sunset Park East,0.300000
2,3143435,Sunset Park West,0.150000
3,2015444,Van Cortlandt Village,0.002273
4,2003900,Mott Haven-Port Morris,0.024615
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.008370
66393,5101137,Westerleigh,0.200000
66394,5101137,Westerleigh,0.200000
66395,5108656,West New Brighton-New Brighton-St. George,0.000833


# **3. average_year_eviction_count_per_nta(nta)**

This measurement measures the average eviction per neighborhood(aggregated by all the evictions happened within a neighborhood and divided by the number of years observed.)

- an example would be: if the result is 120, it means on average, this neighborhood has 120 evictions per year.

$$
\text{average yearly evictions}(\text{neighborhood}) = \frac{\text{Total Evictions}(\text{neighborhood})}{\text{Number of Years Observed}}
$$

In [ ]:
normal[['bin', 'nta', 'average_year_eviction_count_per_nta(nta)']]

,bin,nta,average_year_eviction_count_per_nta(nta)
0,3083989,East New York,266.0
1,3143881,Sunset Park East,38.0
2,3143435,Sunset Park West,39.6
3,2015444,Van Cortlandt Village,172.8
4,2003900,Mott Haven-Port Morris,158.8
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,89.6
66393,5101137,Westerleigh,6.8
66394,5101137,Westerleigh,6.8
66395,5108656,West New Brighton-New Brighton-St. George,89.6


# **4. evictions_per_nta_1kpop(nta)**

This measurement measures the average eviction per neighborhood(aggregated by all the evictions happened within a neighborhood and divided by the number of years observed.) and normalized by every 1000 population.

- an example would be: if the result is 2.74, it means on average, every 1000 people living in this neighborhood, 2.74 of them would get eviction per year.


$$
\text{eviction rate per 1k population}(\text{neighborhood}) = \left( \frac{\text{Total Evictions}(\text{neighborhood})}{\text{Years} \times \text{Population}} \right) \times 1000
$$

In [ ]:
normal[['bin', 'nta', 'evictions_per_nta_1kpop(nta)']]

,bin,nta,evictions_per_nta_1kpop(nta)
0,3083989,East New York,2.747905
1,3143881,Sunset Park East,0.408567
2,3143435,Sunset Park West,0.425770
3,2015444,Van Cortlandt Village,2.122928
4,2003900,Mott Haven-Port Morris,3.578188
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,2.221616
66393,5101137,Westerleigh,0.366241
66394,5101137,Westerleigh,0.366241
66395,5108656,West New Brighton-New Brighton-St. George,3.414764


# **5. evictions_per_1kunit_nta_1kpop(nta)**

This measurement measures the average eviction per unit in each neighborhood(aggregated by all the evictions happened within a neighborhood and divided by all the units) and normalized by every 1000 population and every 1000 units.

- an example would be: if the result is 0.0217, it means every 1000 units in that neighborhood, we observe on average 0.0217 evictions  

$$
\text{eviction rate per 1k population}(\text{neighborhood}) = \left( \frac{\text{Total Evictions}(\text{neighborhood})}{\text{Years} \times \text{Population}} \right) \times 1000
$$

In [ ]:
normal[['bin', 'nta', 'evictions_per_1kunit_nta_1kpop(nta)']]

,bin,nta,evictions_per_1kunit_nta_1kpop(nta)
0,3083989,East New York,1.263405
1,3143881,Sunset Park East,1.502085
2,3143435,Sunset Park West,0.724098
3,2015444,Van Cortlandt Village,0.024717
4,2003900,Mott Haven-Port Morris,0.170431
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.018259
66393,5101137,Westerleigh,6.104020
66394,5101137,Westerleigh,6.104020
66395,5108656,West New Brighton-New Brighton-St. George,0.053090


# **6. average_year_eviction_per_building_nta(nta)**

This measurement measures the average evcitions per 10 building within particular neighborhood.

- an example would be: if the result for an nta is 3.668966, it means, for this particular neighborhood, every 10 buildings, they have 3.668966 evictions on average per year.

$$
\text{evictions per year per 10 buildings}(\text{neighborhood}) = \left( \frac{\text{Total Evictions}(\text{neighborhood})}{\text{Years} \times \text{Total Buildings}(\text{neighborhood})} \right) \times 10
$$

In [ ]:
normal[['bin', 'nta', 'average_year_eviction_per_building_nta(nta)']]

,bin,nta,average_year_eviction_per_building_nta(nta)
0,3083989,East New York,3.668966
1,3143881,Sunset Park East,2.794118
2,3143435,Sunset Park West,2.693878
3,2015444,Van Cortlandt Village,7.081967
4,2003900,Mott Haven-Port Morris,4.916409
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,3.343284
66393,5101137,Westerleigh,2.266667
66394,5101137,Westerleigh,2.266667
66395,5108656,West New Brighton-New Brighton-St. George,3.343284


# **7. children_impacted_nta_per_1kpop**

This measurement measures that for each neighborhood, for every 1000 populations, how many children (under 17) were affected by evictions per year.

- for example, if an nta has 0.618279, this means, for every 1000 people living this neighborhood, about 0.618279 child was affected by evictions.

$$
\text{children impacted per 1k pop}(\text{neighborhood}) = \left ( \text{evictions per 1k}(\text{neighborhood}) \times \frac {\text{children pct}(\text{neighborhood})} {100} \right)
$$

In [ ]:
normal[['bin', 'nta', 'children_impacted_nta_per_1kpop']]

,bin,nta,children_impacted_nta_per_1kpop
0,3083989,East New York,0.618279
1,3143881,Sunset Park East,0.103776
2,3143435,Sunset Park West,0.108146
3,2015444,Van Cortlandt Village,0.560453
4,2003900,Mott Haven-Port Morris,1.005471
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.459875
66393,5101137,Westerleigh,0.086799
66394,5101137,Westerleigh,0.086799
66395,5108656,West New Brighton-New Brighton-St. George,0.877594


# **8. elderly_impacted_nta_per_1kpop**

This measurement measures that for each neighborhood, for every 1000 populations, how many elderly (over 65) were affected by evictions per year.

- for example, if an nta has 0.379211, this means, for every 1000 people living this neighborhood, about 0.379211 elderly was affected by evictions.

$$
\text{elderly impacted per 1k pop}(\text{neighborhood}) = \left ( \text{evictions per 1k}(\text{neighborhood}) \times \frac {\text{elderly pct}(\text{neighborhood})} {100} \right)
$$

In [ ]:
normal[['bin', 'nta', 'elderly_impacted_nta_per_1kpop']]

,bin,nta,elderly_impacted_nta_per_1kpop
0,3083989,East New York,0.379211
1,3143881,Sunset Park East,0.053522
2,3143435,Sunset Park West,0.055776
3,2015444,Van Cortlandt Village,0.237768
4,2003900,Mott Haven-Port Morris,0.361397
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.346572
66393,5101137,Westerleigh,0.037357
66394,5101137,Westerleigh,0.037357
66395,5108656,West New Brighton-New Brighton-St. George,0.430260


# **9. black + hispanic _impacted_nta_per_1kpop**

This measurement measures that for each neighborhood, for every 1000 populations, how many black and hispanic population were affected by evictions per year.

- for example, if an nta has 2.437392, this means, for every 1000 people living this neighborhood, about 2.437392 black and hispanic population was affected by evictions.

$$
\text{black and hispanic impacted per 1k pop}(\text{neighborhood}) = \left ( \text{evictions per 1k}(\text{neighborhood}) \times \frac {\text{black and hispanic population pct}(\text{neighborhood})} {100} \right)
$$

In [ ]:
normal[['bin', 'nta', 'bh_impacted_nta_per_1kpop']]

,bin,nta,bh_impacted_nta_per_1kpop
0,3083989,East New York,2.437392
1,3143881,Sunset Park East,0.174050
2,3143435,Sunset Park West,0.181378
3,2015444,Van Cortlandt Village,1.987061
4,2003900,Mott Haven-Port Morris,3.406435
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,1.026387
66393,5101137,Westerleigh,0.232929
66394,5101137,Westerleigh,0.232929
66395,5108656,West New Brighton-New Brighton-St. George,1.673234


# **10. unemployed _impacted_nta_per_1kpop**

This measurement measures that for each neighborhood, for every 1000 populations, how many unemployed were affected by evictions per year.

- for example, if an nta has 0.305018, this means, for every 1000 people living this neighborhood, about 0.305018 unemployed population was affected by evictions.

$$
\text{unemployed impacted per 1k pop}(\text{neighborhood}) = \left ( \text{evictions per 1k}(\text{neighborhood}) \times \frac {\text{unemployed population pct}(\text{neighborhood})} {100} \right)
$$

In [ ]:
normal[['bin', 'nta', 'unemployed_impacted_nta_per_1kpop']]

,bin,nta,unemployed_impacted_nta_per_1kpop
0,3083989,East New York,0.305018
1,3143881,Sunset Park East,0.030643
2,3143435,Sunset Park West,0.031933
3,2015444,Van Cortlandt Village,0.246260
4,2003900,Mott Haven-Port Morris,0.447274
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.157735
66393,5101137,Westerleigh,0.028567
66394,5101137,Westerleigh,0.028567
66395,5108656,West New Brighton-New Brighton-St. George,0.143420


# **11. burdened_househould_impacted_nta_per_1kpop**

Any household who spend more than 30% of income on housing costs are considered housing-burdened.

This measurement measures that for each neighborhood, for every 1000 population, how many households that already spent more than 30% of income on housing costs were affected by evictions per year.

- for example, if an nta has 1.228314, this means, for every 1000 people living this neighborhood, about 1.228314 hburd households was affected by evictions.

$$
\text{hburd impacted per 1k pop}(\text{neighborhood}) = \left ( \text{evictions per 1k}(\text{neighborhood}) \times \frac {\text{hburd population pct}(\text{neighborhood})} {100} \right)
$$

In [ ]:
normal[['bin', 'nta', 'hburd_impacted_nta_per_1kpop']]

,bin,nta,hburd_impacted_nta_per_1kpop
0,3083989,East New York,1.228314
1,3143881,Sunset Park East,0.178135
2,3143435,Sunset Park West,0.185636
3,2015444,Van Cortlandt Village,1.203700
4,2003900,Mott Haven-Port Morris,1.857080
...,...,...,...
66392,5108502,West New Brighton-New Brighton-St. George,0.715360
66393,5101137,Westerleigh,0.129283
66394,5101137,Westerleigh,0.129283
66395,5108656,West New Brighton-New Brighton-St. George,0.956134
